In [5]:
# Ensure kagglehub is installed
try:
    import kagglehub  # noqa: F401
    print("kagglehub already installed")
except Exception:
    print("Installing kagglehub...")
    %pip install -q kagglehub
    import kagglehub  # noqa: F401
    print("kagglehub installed")

Installing kagglehub...

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
kagglehub installed


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Ensure torch and torchvision are installed
try:
    import torch  # noqa: F401
    import torchvision  # noqa: F401
    print("torch/torchvision already installed")
except Exception:
    print("Installing torch and torchvision (default wheels)...")
    try:
        %pip install -q torch torchvision
    except Exception:
        print("Default install failed; trying CPU wheels index...")
        %pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cpu
    import torch  # noqa: F401
    import torchvision  # noqa: F401
    print("torch/torchvision installed")

torch/torchvision already installed


In [11]:
# Install and download dataset via KaggleHub, then wire into config and smoke test loaders
# If running in a fresh environment, uncomment the next line:
# !pip install kagglehub

import os
from pathlib import Path
import kagglehub

from utils.config import Config
from utils.data import get_dataloaders

# 1) Download the dataset
path = kagglehub.dataset_download("aryankaushik005/custom-dataset")
print("KaggleHub dataset path:", path)

# 2) Find the directory that contains both real_images and fake_images
root = Path(path)
# KaggleHub may place files under a nested 'content' or similar folder; search for the class dirs
candidates = []
for p in [root, *root.rglob("*")]:
    if p.is_dir():
        real = p / "real_images"
        fake = p / "fake_images"
        if real.is_dir() and fake.is_dir():
            candidates.append(p)

if not candidates:
    raise FileNotFoundError("Could not locate a directory containing both 'real_images' and 'fake_images'. Inspect the downloaded path and adjust.")

# Choose the deepest match to avoid picking a higher-level folder accidentally
candidates = sorted(candidates, key=lambda x: len(str(x)))
data_dir = str(candidates[-1])
print("Detected dataDir:", data_dir)

# 3) Update project config to point to this dataDir and save
cfg = Config().load("configs/config.json")
cfg.dataset.dataDir = data_dir
cfg.dataset.trainDir = ""
cfg.dataset.valDir = ""
if not hasattr(cfg.dataset, "trainValSplit"):
    cfg.dataset.trainValSplit = 0.8
if not hasattr(cfg.dataset, "numWorkers"):
    cfg.dataset.numWorkers = 4
if not hasattr(cfg.dataset, "augment"):
    cfg.dataset.augment = True
cfg.overwrite()
print("Updated configs/config.json -> dataset.dataDir:", cfg.dataset.dataDir)

# 4) Smoke test: create loaders and fetch one batch
loaders = get_dataloaders(cfg)
xb, yb = next(iter(loaders["train"]))
print("Train batch shape:", tuple(xb.shape))
print("First labels:", yb[:8].tolist())
print("Classes:", loaders["classes"])

100%|██████████| 4.76G/4.76G [03:31<00:00, 24.2MB/s]

Extracting files...


KaggleHub dataset path: /Users/tony/.cache/kagglehub/datasets/aryankaushik005/custom-dataset/versions/1
Detected dataDir: /Users/tony/.cache/kagglehub/datasets/aryankaushik005/custom-dataset/versions/1/content
Updated configs/config.json -> dataset.dataDir: /Users/tony/.cache/kagglehub/datasets/aryankaushik005/custom-dataset/versions/1/content
Train batch shape: (32, 3, 256, 256)
First labels: [1, 1, 1, 0, 1, 1, 0, 0]
Classes: {0: 'fake_images', 1: 'real_images'}


In [12]:
# Quick training sanity check (1 short epoch)
import torch
from utils.config import Config
from utils.data import get_dataloaders
from utils.model import PCAFormer

cfg = Config().load("configs/config.json")
loaders = get_dataloaders(cfg)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PCAFormer(cfg.model).to(device)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

model.train()
max_train_steps = 100  # keep short for sanity
running = 0.0
steps = 0
for xb, yb in loaders["train"]:
    xb = xb.to(device)
    yb = yb.float().unsqueeze(1).to(device)

    optimizer.zero_grad()
    out = model(xb)
    loss = criterion(out, yb)
    loss.backward()
    optimizer.step()

    running += loss.item()
    steps += 1
    if steps >= max_train_steps:
        break

print(f"Avg train loss over {steps} steps: {running/max_train_steps:.4f}")

# Simple val accuracy on a few batches
model.eval()
correct = 0
count = 0
max_val_steps = 50
with torch.no_grad():
    for xb, yb in loaders["val"]:
        xb = xb.to(device)
        yb = yb.to(device)
        prob = model(xb).squeeze(1)
        pred = (prob >= 0.5).long().cpu()
        correct += (pred == yb.cpu()).sum().item()
        count += yb.numel()
        max_val_steps -= 1
        if max_val_steps <= 0:
            break

print(f"Val accuracy (subset): {correct / max(1, count):.3f}")

_LinAlgError: linalg.svd: (Batch element 0): The algorithm failed to converge because the input matrix contained non-finite values.